In [1]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16013712768889670522, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7059054196427073471
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4526794470233964818
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9683895382690461522
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Streaming output truncated to the last 5000 lines.
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_16_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_17_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_181_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_182_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_183_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_184_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_185_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_186_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_187_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_188_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_189_100.jpg  
  inflating: /co

In [0]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from glob import glob
import numpy as np

In [6]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['test-multiple_fruits', 'papers', 'Training', 'LICENSE', 'readme.md', 'Test']

In [7]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [8]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [9]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Grape Pink          492
Eggplant            468
Nectarine Flat      480
Apple Golden 2      492
Apple Red 3         429
Grape White 3       492
Maracuja            490
Plum 3              900
Kohlrabi            471
Potato Red Washed   453
Potato White        450
Physalis with Husk  492
Tomato 2            672
Ginger Root         99
Pear Abate          490
Grapefruit Pink     490
Lemon               492
Potato Sweet        450
Cantaloupe 1        492
Apple Pink Lady     456
Pear Forelle        702
Pear Red            666
Cherry Wax Black    492
Orange              479
Apple Golden 1      492
Apple Golden 3      481
Redcurrant          492
Carambula           490
Tomato 3            738
Apple Red 1         492
Chestnut            450
Kiwi                466
Grape White         490
Cherry 1            492
Dates               490
Apple Crimson Snow  444
Cantaloupe 2        492
Apple Red Yellow 1  492
Nut Pecan           178
Banana Red     

In [10]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2, data_format='channels_last')
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, data_format='channels_last')
test_datagenerator = ImageDataGenerator(rescale= 1./255, data_format='channels_last')

In [12]:
#Creating Batches
image_size = (256, 256)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [13]:
#useful imports
import pickle
!pip install scikit-optimize
import skopt
from skopt.utils import use_named_args
from skopt.callbacks import CheckpointSaver
from collections import namedtuple

     |████████████████████████████████| 81kB 2.1MB/s 


In [14]:
#set up saving. If old data are present, they are loaded
Hyperpars = namedtuple('Hyperpars', ['learning_rate', 'alpha', 'dropout'])
HPO_save_path = '/content/drive/My Drive/Progetto Advanced Machine Learning/Model Checkpoints/'
dict_save_path = HPO_save_path + 'Custom Model HPO History Dictionary.pickle'
checkpoint_save_path = HPO_save_path + 'Custom Model HPO Checkpoint.pickle'
if os.path.isfile(dict_save_path):
  with open(dict_save_path, 'rb') as f:
    print('Loading previous history...')
    history = pickle.load(f)
else:
  print('No previous history found')
  history = dict()
if os.path.isfile(checkpoint_save_path):
  with open(checkpoint_save_path, 'rb') as f:
    print('Loading previous results...')
    results = skopt.load(checkpoint_save_path)
else:
  print('No previous results found')
checkpoint_saver = CheckpointSaver(checkpoint_save_path)

No previous history found
No previous results found


In [0]:
#set random seeds to get reproducible results
np.random.seed(1)
tf.random.set_seed(2)

def create_model(learning_rate, alpha, dropout):
  model_path = '/content/drive/My Drive/Progetto Advanced Machine Learning/Model Checkpoints/Custom Model 1 2020-06-01 10:56:21.010759.hdf5'
  model = tf.keras.models.load_model(model_path)
  x = model.layers[-2].output
  x = Dropout(dropout)(x)
  predictions = Dense(120, activation='softmax', name='prediction', kernel_regularizer=tf.keras.regularizers.l2(alpha))(x)
  model = Model(inputs=model.inputs, outputs=predictions)
  for layer in model.layers[:-2]:
    layer.trainable = False
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy']) 
  return model

#declare the search space
SEARCH_SPACE = [skopt.space.Real(0.0001, 0.1, name='learning_rate', prior='log-uniform'),
                skopt.space.Real(1e-9, 1, name='alpha', prior='log-uniform'),
                skopt.space.Real(0.0001, 0.95, name='dropout', prior='log-uniform')]

@use_named_args(SEARCH_SPACE)
def fitness(learning_rate, alpha, dropout):
  print('ε: {}, ɑ {}, dropout {}'.format(learning_rate, alpha, dropout))
  hyperpars = Hyperpars(learning_rate, alpha, dropout)
  print(len(history))
  if hyperpars in history:
    print('REPEATING POINTS!!!')
  model = create_model(learning_rate, alpha, dropout)
  #compile generators
  train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
  val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', shuffle=False, seed = 20052020)
  #train
  early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
  training_results = model.fit(train_batches, epochs=5, verbose=1, shuffle=True, validation_data=val_batches, workers=2)
  history[hyperpars] = training_results.history
  with open(dict_save_path, 'wb') as f:
    pickle.dump(history, f)
  return training_results.history['val_accuracy'][-1]

In [0]:
#if we have old results, do a warm start. If not, cold start
try:
  known_xs = results.x_iters
  known_ys = results.func_vals
  result = skopt.forest_minimize(fitness, SEARCH_SPACE, n_calls=10, callback=checkpoint_saver, x0=known_xs, y0=known_ys)
except NameError:
  result = skopt.forest_minimize(fitness, SEARCH_SPACE, n_calls=10, callback=checkpoint_saver)

ε: 0.0980741228990046, ɑ 0.24718165404756284, dropout 0.00032332824201601514
0
Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Epoch 1/5
1514/1514 [==============================] - 789s 521ms/step - loss: 52.7149 - accuracy: 0.1217 - val_loss: 46.8876 - val_accuracy: 0.1290
Epoch 2/5
1514/1514 [==============================] - 794s 524ms/step - loss: 52.2333 - accuracy: 0.1272 - val_loss: 62.1621 - val_accuracy: 0.0941
Epoch 3/5
1514/1514 [==============================] - 798s 527ms/step - loss: 51.2171 - accuracy: 0.1309 - val_loss: 50.7462 - val_accuracy: 0.1368
Epoch 4/5
1514/1514 [==============================] - 809s 535ms/step - loss: 51.5689 - accuracy: 0.1317 - val_loss: 49.8214 - val_accuracy: 0.1289
Epoch 5/5
1514/1514 [==============================] - 812s 536ms/step - loss: 51.4066 - accuracy: 0.1318 - val_loss: 51.6817 - val_accuracy: 0.1374
ε: 0.0029464233838192235, ɑ 0.09116292059447313, dropout 0.08832308429673111
1
Found 4

In [0]:
fig1 = skopt.plots.plot_evaluation(result)
fig1

In [0]:
fig2 = skopt.plots.plot_objective(result)
fig2

In [0]:
len(history)